In [233]:
import numpy as np
import os
import torchvision
from PIL import Image
import cv2
from skimage import color
from skimage import io
from tqdm import tqdm
from glob import glob
import torch
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip

### Data Augmentation

In [654]:
np_dir = "/dtu/blackhole/11/173553/carseg_data/train"
bg_dir = "/dtu/blackhole/11/173553/carseg_data/carseg_raw_data/landscape/images"

Add Background

In [655]:
def get_random_bg():
    bg_dir = "/dtu/blackhole/11/173553/carseg_data/carseg_raw_data/landscape/images"
    bgs_path = os.listdir(bg_dir)    
    while True:
        idx = np.random.randint(low=0, high=len(bgs_path))
        bg_path = os.path.join(bg_dir,bgs_path[idx])
        bg = cv2.imread(bg_path)
        resized_bg = cv2.resize(bg,(np_img.shape[1], np_img.shape[2]), interpolation = cv2.INTER_AREA)
        return resized_bg

Image Augmentation

In [665]:
def augment_data(img):
    
    H = 256
    W = 256

    x = np.transpose(img[:3]*255,(1,2,0)).astype(np.uint8)
    y = img[3]

    np.random.seed(0)
    mod = np.random.randint(low=0, high=5)

    if mod==0:
        aug = CenterCrop(H, W, p=1.0)
        augmented = aug(image=x, mask=y)
        x_aug = augmented["image"]
        y_aug = augmented["mask"]

    if mod==1:
        aug = RandomRotate90(p=1.0)
        augmented = aug(image=x, mask=y)
        x_aug = augmented['image']
        y_aug = augmented['mask']

    if mod==2:
        aug = GridDistortion(p=1.0)
        augmented = aug(image=x, mask=y)
        x_aug = augmented['image']
        y_aug = augmented['mask']

    if mod==3:
        aug = HorizontalFlip(p=1.0)
        augmented = aug(image=x, mask=y)
        x_aug = augmented['image']
        y_aug = augmented['mask']

    if mod==4:
        aug = VerticalFlip(p=1.0)
        augmented = aug(image=x, mask=y)
        x_aug = augmented['image']
        y_aug = augmented['mask']

    i = cv2.resize(x_aug, (W, H))
    i = np.transpose(i/255,(2,0,1))
    m = cv2.resize(y_aug, (W, H))
    m = np.reshape(m,(1,256,256))
    res = np.concatenate((i,m),axis=0)

    return res,mod

In [666]:
def get_file(dir):
    listOfFile = os.listdir(dir)
    allFiles = []
    for entry in listOfFile:
        fullPath = os.path.join(dir, entry)
        allFiles.append(fullPath)
    return allFiles

add random background and one type of data augmentation

In [668]:
nps_path = os.listdir(np_dir)
for np_path in tqdm(nps_path):
    np_img = np.load(os.path.join(np_dir,np_path))
    bg = get_random_bg()
    np_img[:3] *= 255
    for i in range(np_img.shape[1]):
        for j in range(np_img.shape[2]):
            if np_img[3][i][j]==0:
                np_img[0][i][j] = bg[i][j][0]
                np_img[1][i][j] = bg[i][j][1]
                np_img[2][i][j] = bg[i][j][2]
    np_img[:3] /= 255
    res,mod = augment_data(np_img)
    name = os.path.join(np_dir,np_path)[:-4]+"_aug_"+str(mod)+".npy"
    np.save(name,res)

In [675]:
nps_path = os.listdir(np_dir)
len(nps_path)

6988

show image from numpy array

In [561]:
def show_img_from_np(x):
    if(type(x)==np.ndarray):
        np_data = x
    else:
        np_data = np.load(x)
    img_from_np = np.transpose(np_data[:3]*255,(1,2,0)).astype(np.uint8)
    img = Image.fromarray(img_from_np,mode="RGB")
    img.save('my.png')
    img.show()

In [681]:
show_img_from_np("/dtu/blackhole/11/173553/carseg_data/train/332_aug_2.npy")

START /usr/bin/eog "/tmp/tmplbny8_q5.PNG"



(eog:16667): Gtk-WARNING **: 17:03:38.655: cannot open display: 


In [617]:
show_img_from_np("/dtu/blackhole/11/173553/carseg_data/clean_data/0_addbg.npy")

START /usr/bin/eog "/tmp/tmpf0h8rskh.PNG"



(eog:10380): Gtk-WARNING **: 15:03:19.383: cannot open display: 


show mask from numpy array

In [648]:
label = np.load("/dtu/blackhole/11/173553/carseg_data/clean_data/1_aug_2.npy")[3]
label = np.reshape(label,(1,256,256))
a = np.zeros((1,256,256))
final_img = np.concatenate((a,a,label),axis=0)

img = Image.fromarray(np.transpose(final_img*25,(1,2,0)).astype(np.uint8),mode="RGB")
img.save('my.png')
img.show()

### Map color and label (useless)

In [ ]:
img = Image.open("/dtu/blackhole/11/173553/carseg_data/clean_data/1_a.npy")
img

In [309]:
np_img = np.array(img)
np_img.shape

(256, 512, 3)

In [311]:
colors = [[19,100,19],
         [15,248,250],
         [20,19,251],
         [249,249,10],
         [250,149,10],
         [149,7,149], 
         [249,9,250],
         [8,249,9],
         [117,31,0]]

def img2mask(np_img):
    threshold = 50
    new_np = np.zeros((np_img.shape[0],np_img.shape[1]))
    for i in range(np_img.shape[0]):
        for j in range(np_img.shape[1]):
            for k, color in enumerate(colors):
                # if np_img[i][j][0]==color[0] and np_img[i][j][1]==color[1] and abs(np_img[i][j][2]==color[2]):
                if not np_img[i][j][2]==np_img[i][j][1]==np_img[i][j][0]:
                    if abs(np_img[i][j][0]-color[0])<=threshold and abs(np_img[i][j][1]-color[1])<=threshold and abs(np_img[i][j][2]-color[2])<=threshold:
                       new_np[i,j] = k+1
    return new_np

def mask2img(mask):
    new_np = np.zeros((mask.shape[0],mask.shape[1],3))
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i][j]:
                new_np[i][j] = colors[int(mask[i][j]-1)]
    return new_np

In [312]:
a = img2mask(np_img)
b = mask2img(a)
a.shape,b.shape 

((256, 512), (256, 512, 3))

In [313]:
img = Image.fromarray(b.astype(np.uint8))
img.save('my.png')
img.show()

START /usr/bin/eog "/tmp/tmpmly21c_m.PNG"



(eog:731): Gtk-WARNING **: 06:27:40.205: cannot open display: 
